x1 x2 x3 x4 x5 x6 x7
去掉沒有甚麼關係的自變量

1. ALL-in 所有的東西都丟進去 
  - prior knowledge
  - you have to
  - prepare Backward Elimination
---
2. Backward Elimination 反向淘汰
  - select a significance level to stay in the model(e.g. SL=0.05 該變量是否對模型有顯著影響)
  - fit all predictors
  - consider the predictor with the highest P-value.
  - if P > SL remove the predictor
  - fit model without this predictor
---
3. Forward Selection 順向選擇
  - select a significance level to stay in the model(e.g. SL=0.05 該變量是否對模型有顯著影響)
  - fit all simple regression models y ~ xn select the one with the lowest P-value
  - keep this variable and fit all possible models with one exta predictor added to the one(s) you already have
  - consider the predictor with the lowest P-value if p < SL go to pre step otherwise go to fin

4. Bidirectional Elmination 雙向淘汰
---
  - select a signficance level to enter and to stay in the model e.g.:slenter = 0.05 slstay = 0.05
  - perform the next step of Forward Selection (new variables must have: P < slenter to enter)
  - perform all steprs of backward Elimination (old variables must have p < slstay to stay)
  - no new variables can enter and no old variables can exit
5. Score Comparion 訊息量比較
---
  - select a criterion of goodness of fit(e.g. Akaike criterion)
  - construct all possible regression models: 2^n - 1 total combination (10 columns mean 1023 models)
  - select the on with the best criterion

2,3,4 -> 逐步回歸

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('./multiple_linear_regression_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


In [3]:
data.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# label_encoder_x = LabelEncoder()
# x[:, -1] = label_encoder_x.fit_transform(x[:, -1])
x[:5]

array([[165349.2, 136897.8, 471784.1, 'New York'],
       [162597.7, 151377.59, 443898.53, 'California'],
       [153441.51, 101145.55, 407934.54, 'Florida'],
       [144372.41, 118671.85, 383199.62, 'New York'],
       [142107.34, 91391.77, 366168.42, 'Florida']], dtype=object)

In [5]:
ct = ColumnTransformer([("State", OneHotEncoder(), [3])], remainder='passthrough')  # 虛擬編碼 產生共線性 D2 = 1 - D1 所以要去掉D2 -> 要顯示 總編碼數量 - 1
x = ct.fit_transform(x)
x = x[:, 1:] #avoid the dummy varibale trap
x[:5]

array([[0.0, 1.0, 165349.2, 136897.8, 471784.1],
       [0.0, 0.0, 162597.7, 151377.59, 443898.53],
       [1.0, 0.0, 153441.51, 101145.55, 407934.54],
       [0.0, 1.0, 144372.41, 118671.85, 383199.62],
       [1.0, 0.0, 142107.34, 91391.77, 366168.42]], dtype=object)

In [6]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((40, 5), (10, 5), (40,), (10,))

In [7]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(x_train, y_train)
y_pred = reg.predict(x_test) # all in
reg.score(x_test, y_test)

0.9347068473282966

In [8]:
import statsmodels.formula.api as sm

x_train = np.append(arr=np.ones((40, 1)), values=x_train, axis=1) # y = b + wx 要把b當係數 -> b*1 

In [9]:
x_train[:5]

array([[1.0, 1.0, 0.0, 55493.95, 103057.49, 214634.81],
       [1.0, 0.0, 1.0, 46014.02, 85047.44, 205517.64],
       [1.0, 1.0, 0.0, 75328.87, 144135.98, 134050.07],
       [1.0, 0.0, 0.0, 46426.07, 157693.92, 210797.67],
       [1.0, 1.0, 0.0, 91749.16, 114175.79, 294919.57]], dtype=object)

In [11]:
import statsmodels.regression.linear_model as sm
x_opt = x_train[:, [0, 1, 2, 3, 4, 5]]
x_opt = np.array(x_opt, dtype=float)
reg_OLS = sm.OLS(endog=y_train, exog=x_opt).fit()
reg_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.943
Method:                 Least Squares   F-statistic:                     129.7
Date:                Mon, 01 May 2023   Prob (F-statistic):           3.91e-21
Time:                        13:30:40   Log-Likelihood:                -421.10
No. Observations:                  40   AIC:                             854.2
Df Residuals:                      34   BIC:                             864.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.255e+04   8358.538      5.091      0.000    2.56e+04    5.95e+04
x1          -959.2842   4038.108     -0.238      0.814   -9165.706    7247.138
x2           699.3691   3661.563      0.191      0.850   -6741.822    8140.560
x3             0.7735      0.055     14.025      0.000       0.661       0.886
x4             0.0329      0.066      0.495      0.624      -0.102       0.168
x5             0.0366      0.019      1.884      0.068      -0.003       0.076
==============================================================================
Omnibus:                       15.823   Durbin-Watson:                   2.468
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.231
Skew:                          -1.094   Prob(JB):                     9.03e-06
Kurtosis:                       6.025   Cond. No.                     1.49e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.49e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [12]:
x_opt = x_train[:, [0, 1, 3, 4, 5]]
x_opt = np.array(x_opt, dtype=float)
reg_OLS = sm.OLS(endog=y_train, exog=x_opt).fit()
reg_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.944
Method:                 Least Squares   F-statistic:                     166.7
Date:                Mon, 01 May 2023   Prob (F-statistic):           2.87e-22
Time:                        13:33:53   Log-Likelihood:                -421.12
No. Observations:                  40   AIC:                             852.2
Df Residuals:                      35   BIC:                             860.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.292e+04   8020.397      5.352      0.000    2.66e+04    5.92e+04
x1         -1272.1608   3639.780     -0.350      0.729   -8661.308    6116.986
x2             0.7754      0.053     14.498      0.000       0.667       0.884
x3             0.0319      0.065      0.488      0.629      -0.101       0.165
x4             0.0363      0.019      1.902      0.065      -0.002       0.075
==============================================================================
Omnibus:                       16.074   Durbin-Watson:                   2.467
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               24.553
Skew:                          -1.086   Prob(JB):                     4.66e-06
Kurtosis:                       6.164   Cond. No.                     1.43e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.43e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
x_opt = x_train[:, [0, 3, 4, 5]]
x_opt = np.array(x_opt, dtype=float)
reg_OLS = sm.OLS(endog=y_train, exog=x_opt).fit()
reg_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     227.8
Date:                Mon, 01 May 2023   Prob (F-statistic):           1.85e-23
Time:                        13:34:12   Log-Likelihood:                -421.19
No. Observations:                  40   AIC:                             850.4
Df Residuals:                      36   BIC:                             857.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.299e+04   7919.773      5.428      0.000    2.69e+04    5.91e+04
x1             0.7788      0.052     15.003      0.000       0.674       0.884
x2             0.0294      0.064      0.458      0.650      -0.101       0.160
x3             0.0347      0.018      1.896      0.066      -0.002       0.072
==============================================================================
Omnibus:                       15.557   Durbin-Watson:                   2.481
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               22.539
Skew:                          -1.081   Prob(JB):                     1.28e-05
Kurtosis:                       5.974   Cond. No.                     1.43e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.43e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
x_opt = x_train[:, [0, 3, 5]]
x_opt = np.array(x_opt, dtype=float)
reg_OLS = sm.OLS(endog=y_train, exog=x_opt).fit()
reg_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.947
Method:                 Least Squares   F-statistic:                     349.0
Date:                Mon, 01 May 2023   Prob (F-statistic):           9.65e-25
Time:                        13:34:47   Log-Likelihood:                -421.30
No. Observations:                  40   AIC:                             848.6
Df Residuals:                      37   BIC:                             853.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.635e+04   2971.236     15.598      0.000    4.03e+04    5.24e+04
x1             0.7886      0.047     16.846      0.000       0.694       0.883
x2             0.0326      0.018      1.860      0.071      -0.003       0.068
==============================================================================
Omnibus:                       14.666   Durbin-Watson:                   2.518
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               20.582
Skew:                          -1.030   Prob(JB):                     3.39e-05
Kurtosis:                       5.847   Cond. No.                     4.97e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.97e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [15]:
x_opt = x_train[:, [0, 3]]
x_opt = np.array(x_opt, dtype=float)
reg_OLS = sm.OLS(endog=y_train, exog=x_opt).fit()
reg_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.945
Model:                            OLS   Adj. R-squared:                  0.944
Method:                 Least Squares   F-statistic:                     652.4
Date:                Mon, 01 May 2023   Prob (F-statistic):           1.56e-25
Time:                        13:34:59   Log-Likelihood:                -423.09
No. Observations:                  40   AIC:                             850.2
Df Residuals:                      38   BIC:                             853.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.842e+04   2842.717     17.032      0.000    4.27e+04    5.42e+04
x1             0.8516      0.033     25.542      0.000       0.784       0.919
==============================================================================
Omnibus:                       13.132   Durbin-Watson:                   2.325
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               16.254
Skew:                          -0.991   Prob(JB):                     0.000295
Kurtosis:                       5.413   Cond. No.                     1.57e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.57e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [29]:
x_1 = data.iloc[:, 0].values
y_1 = data.iloc[:, -1].values
x_1_train, x_1_test, y_1_train, y_1_test = train_test_split(x_1, y_1, test_size=0.2, random_state=0)
x_1_train.shape = (40, 1)
x_1_test.shape = (10, 1)
reg1 = LinearRegression().fit(x_1_train, y_1_train)
y_pred = reg1.predict(x_1_test)
reg1.score(x_1_test, y_1_test)

0.9464587607787219